In [9]:
import pandas as pd
from IPython.display import clear_output 
import re
# from google.colab import drive
# drive.mount('/content/drive/')

In [10]:
df1 = pd.read_csv("/content/drive/MyDrive/CBSA22(Elaine)/real_2020_22tranfinal.csv",nrows=120000)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (26,28,29,31) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# cleansing

In [ ]:
import re
import string

def preprocessing(line):
  # remove special characters
  line = ''.join(c for c in line if c in string.printable)

  # remove Chinese characters
  line = ''.join(c for c in line if c not in '\u4e00\u9fff')

  # remove "===== shared post ====="
  line = re.sub(r'===== shared post =====', '', line)

  # removing header number
  line = re.sub(r'^\s?\d+(.*)$', r'\1', line)
  # removing trailing spaces
  line = line.strip()
  # words may be split between Lines, ensure we Link them back together
  line = re.sub(r'\s?-\s?', '-', line)
  # remove space prior to punctuation
  line = re.sub(r'\s?([,:;\.])', r'\1', line)
  # ESG contains a Lot of figures that are not relevant to grammatical structure
  line = re.sub(r'\d{5,}', r' ', line)
  # remove emails
  line = re.sub(r'\S*@\S*\s?', '', line)
  # remove mentions of URLS
  line = re.sub(r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*', r' ', line)
  # remove multiple spaces
  line = re.sub(r'\s+', ' ', line)
  # join next Line with space
  line = re.sub(r' \n', ' ', line)
  line = re.sub(r' \\ n', ' ', line)
  line = re.sub(r'.\n', '. ', line)
  line = re.sub(r'\x0c', ' ', line)
  line = str(line).strip()
  return line


In [ ]:
import csv

cleanText_headline = []
cleanText_content = []
df1['tran_headline']=df1['tran_headline'].astype(str)
df1['tran_content']=df1['tran_content'].astype(str)
df1[['tran_headline','tran_content']]= df1[['tran_headline','tran_content']].fillna(' ')

exceptions = []
for index in tqdm(range(0,df1.shape[0])):
  try:
    cleanText_headline.append(preprocessing(df1["tran_headline"][index]))
    cleanText_content.append(preprocessing(df1["tran_content"][index]))
  except Exception as e:
    print(e)
    cleanText_headline.append('')
    cleanText_content.append('')
    exceptions.append(e)

df1['cleanText_headline']=cleanText_headline
df1['cleanText_content']=cleanText_content

if exceptions:
  with open('exceptions.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for exception in exceptions:
      writer.writerow([exception])


100%|██████████| 120000/120000 [04:39<00:00, 429.72it/s]


In [ ]:
df1

,Unnamed: 0,Unnamed: 0.1,pubdate,non_view_engagements,comment_count,like_count,dislike_count,love_count,haha_count,wow_count,...,cleanText_headline,cleanText_content,sentiment_headline,sentiment_label_headline,sentiment_content,sentiment_label_content,ESG_label_headline,ESG_label_score_headline,ESG_label_content,ESG_label_score_content
0,0,0,2020-09-10 11:56:07,1,0.0,1.0,0.0,0.0,0.0,0.0,...,"\ n [We, never let Nangsheng encapsulate.] On...","[We, never let Nandosheng encapsulate.] One pe...",0.982091,Neutral,0.967710,Neutral,Pollution & Waste,0.290469,Pollution & Waste,0.254028
1,1,1,2020-05-21 00:23:03,0,0.0,0.0,0.0,0.0,0.0,0.0,...,"[HoneyBee] It turned out that today's ""World B...","[HoneyBee] It turned out that today's ""World B...",0.999991,Neutral,0.999991,Neutral,Natural Capital,0.931946,Natural Capital,0.931946
2,2,2,2021-09-07 14:31:02,47,4.0,41.0,0.0,NaN,NaN,NaN,...,Meet the Director of Anbai Global Finance and ...,Meet the director of Anbai Global Finance And ...,0.999968,Neutral,0.997915,Neutral,Community Relations,0.893131,Community Relations,0.936199
3,3,3,2020-07-20 16:56:49,0,NaN,0.0,NaN,0.0,0.0,0.0,...,nd 'S' of 6S Quality Process 'Sourcing' #6squa...,[#Sustainability x NU SKIN 6s-Find the best so...,0.998486,Neutral,0.986318,Neutral,Product Liability,0.369337,Pollution & Waste,0.941735
4,4,4,2020-07-20 21:14:41,0,NaN,0.0,NaN,0.0,0.0,0.0,...,The 3rd 'S' of 6S Quality Process 'Specificati...,The 3rd 's' of 6s Quality Process 'Specificati...,0.997296,Neutral,0.996695,Neutral,Product Liability,0.635044,Product Liability,0.691391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,54995,119996,2021-04-16 18:30:00,8,NaN,7.0,0.0,NaN,NaN,NaN,...,"""Meteorological Cold Knowledge"": Strong Thunde...","""Meteorological Cold Knowledge"": Strong Thunde...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119996,54996,119997,2021-04-21 18:30:00,5,NaN,4.0,0.0,NaN,NaN,NaN,...,"[""Meteorological Cold Knowledge"": How far is t...","[""Meteorological Cold Knowledge"": How far is t...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119997,54997,119998,2021-04-30 18:30:00,0,NaN,0.0,0.0,NaN,NaN,NaN,...,"""Meteorological Cold Knowledge"": Thai-style cr...","""Meteorological Cold Knowledge"": Thai-style cr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119998,54998,119999,2021-05-03 18:30:03,2,NaN,1.0,0.0,NaN,NaN,NaN,...,"""Meteorological Cold Knowledge"": Wind Memorand...","""Meteorological Cold Knowledge"": Wind Memorand...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df2 = pd.read_csv("/content/drive/MyDrive/CBSA22(Elaine)/real_2020_22tranfinal.csv")

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (23,24,26,28,29,31) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,Unnamed: 0.1,pubdate,non_view_engagements,comment_count,like_count,dislike_count,love_count,haha_count,wow_count,...,cleanText_headline,cleanText_content,sentiment_headline,sentiment_label_headline,sentiment_content,sentiment_label_content,ESG_label_headline,ESG_label_score_headline,ESG_label_content,ESG_label_score_content
0,0,0,2020-09-10 11:56:07,1,0.0,1.0,0.0,0.0,0.0,0.0,...,shared post never let nangsheng encapsulate on...,shared post never let nandosheng encapsulate o...,0.982091,Neutral,0.967710,Neutral,Pollution & Waste,0.290469,Pollution & Waste,0.254028
1,1,1,2020-05-21 00:23:03,0,0.0,0.0,0.0,0.0,0.0,0.0,...,honeybee turned today world bee day guracin be...,honeybee turned today world bee day guracin be...,0.999991,Neutral,0.999991,Neutral,Natural Capital,0.931946,Natural Capital,0.931946
2,2,2,2021-09-07 14:31:02,47,4.0,41.0,0.0,NaN,NaN,NaN,...,meet director anbai global finance reach follo...,meet director anbai global finance reach follo...,0.999968,Neutral,0.997915,Neutral,Community Relations,0.893131,Community Relations,0.936199
3,3,3,2020-07-20 16:56:49,0,NaN,0.0,NaN,0.0,0.0,0.0,...,quality process sourcing squalityprocess,shared post sustainability skin find best sour...,0.998486,Neutral,0.986318,Neutral,Product Liability,0.369337,Pollution & Waste,0.941735
4,4,4,2020-07-20 21:14:41,0,NaN,0.0,NaN,0.0,0.0,0.0,...,quality process specification,quality process specification want best red he...,0.997296,Neutral,0.996695,Neutral,Product Liability,0.635044,Product Liability,0.691391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239997,24996,239996,2022-04-05 16:31:54,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239998,24997,239997,2022-04-05 16:31:00,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239999,24998,239998,2022-04-05 16:31:54,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
240000,24999,239999,2022-04-05 21:08:12,0,0.0,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df2 = df2[120000:]

In [ ]:
df2

,Unnamed: 0,Unnamed: 0.1,pubdate,non_view_engagements,comment_count,like_count,dislike_count,love_count,haha_count,wow_count,...,cleanText_headline,cleanText_content,sentiment_headline,sentiment_label_headline,sentiment_content,sentiment_label_content,ESG_label_headline,ESG_label_score_headline,ESG_label_content,ESG_label_score_content
120000,55000,120001,2021-05-10 14:00:00,0,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120001,55001,120002,2021-05-12 18:00:52,1,NaN,1.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120002,55002,120003,2021-05-14 14:00:00,4,0.0,3.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120003,55003,120004,2021-05-21 18:30:18,0,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120004,55004,120005,2021-05-24 15:00:52,0,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239997,24996,239996,2022-04-05 16:31:54,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239998,24997,239997,2022-04-05 16:31:00,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239999,24998,239998,2022-04-05 16:31:54,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
240000,24999,239999,2022-04-05 21:08:12,0,0.0,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import csv
from tqdm import tqdm 

cleanText_headline = []
cleanText_content = []
# df2['tran_headline']=df2['tran_headline'].astype(str)
# df2['tran_content']=df2['tran_content'].astype(str)
# df2[['tran_headline','tran_content']]= df2[['tran_headline','tran_content']].fillna(' ')

exceptions = []
for index in tqdm(range(120000,240002)):
  try:
    cleanText_headline.append(preprocessing(df2["tran_headline"][index]))
    cleanText_content.append(preprocessing(df2["tran_content"][index]))
  except Exception as e:
    print(f"Error occurred at index {index}: {e}")
    cleanText_headline.append('')
    cleanText_content.append('')
    exceptions.append(e)

df2['cleanText_headline']=cleanText_headline
df2['cleanText_content']=cleanText_content

if exceptions:
  with open('exceptions.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for exception in exceptions:
      writer.writerow([exception])


100%|██████████| 120002/120002 [02:52<00:00, 696.99it/s]
<ipython-input-86-a2d5630cbf43>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['cleanText_headline']=cleanText_headline
<ipython-input-86-a2d5630cbf43>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['cleanText_content']=cleanText_content


In [ ]:
df2

,Unnamed: 0,Unnamed: 0.1,pubdate,non_view_engagements,comment_count,like_count,dislike_count,love_count,haha_count,wow_count,...,cleanText_headline,cleanText_content,sentiment_headline,sentiment_label_headline,sentiment_content,sentiment_label_content,ESG_label_headline,ESG_label_score_headline,ESG_label_content,ESG_label_score_content
120000,55000,120001,2021-05-10 14:00:00,0,NaN,0.0,0.0,NaN,NaN,NaN,...,"[""Meteorological Cold Knowledge"": No. 3 has a ...","[""Meteorological Cold Knowledge"": No. 3 has a ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120001,55001,120002,2021-05-12 18:00:52,1,NaN,1.0,0.0,NaN,NaN,NaN,...,"[""Meteorological Cold Knowledge"": The closer t...","[""Meteorological Cold Knowledge"": The closer t...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120002,55002,120003,2021-05-14 14:00:00,4,0.0,3.0,0.0,0.0,0.0,0.0,...,[Hot Face] [The hot weather should prevent hea...,[Hot Face] [The hot weather should prevent hea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120003,55003,120004,2021-05-21 18:30:18,0,NaN,0.0,0.0,NaN,NaN,NaN,...,"[""Meteorological Cold Knowledge"": significant ...","[""Meteorological Cold Knowledge"": significant ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120004,55004,120005,2021-05-24 15:00:52,0,NaN,0.0,0.0,NaN,NaN,NaN,...,"""Meteorological Cold Knowledge"": The most [WIN...","""Meteorological Cold Knowledge"": The most [WIN...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239997,24996,239996,2022-04-05 16:31:54,0,NaN,NaN,NaN,NaN,NaN,NaN,...,Australia's blue mountain mud pouring in Briti...,"The blue mountains west of Sydney, the largest...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239998,24997,239997,2022-04-05 16:31:00,0,NaN,NaN,NaN,NaN,NaN,NaN,...,Australia's blue mountain mud pouring in Briti...,"The blue mountains west of Sydney, the largest...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239999,24998,239998,2022-04-05 16:31:54,0,NaN,NaN,NaN,NaN,NaN,NaN,...,Australia's blue mountain mud pouring in Briti...,Rescuers sent helicopters to The blue mountain...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
240000,24999,239999,2022-04-05 21:08:12,0,0.0,0.0,0.0,NaN,NaN,NaN,...,Water deposits in ponds are less than six poin...,The rainwater has not increased in April. The ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_total = pd.concat([df1, df2],axis=0)
df_total.drop(columns=["Unnamed: 0","Unnamed: 0.1"],inplace=True)

In [ ]:
df_total.to_csv("/content/drive/MyDrive/CBSA22(Elaine)/real_2020_22tranfinal.csv")

# Install Transoformers for ESG labelling

In [11]:
!pip install transformers
clear_output()

In [12]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

finbertESG = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-esg-9-categories',num_labels=9)
tokenizerESG = BertTokenizer.from_pretrained('yiyanghkust/finbert-esg-9-categories')
nlpESG = pipeline("text-classification", model=finbertESG, tokenizer=tokenizerESG, max_length=512, truncation=True)
clear_output()

In [13]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
import warnings
warnings.simplefilter('ignore')

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

Downloading:   0%|          | 0.00/747 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [14]:
# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]
clear_output()
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

# the thing we needa run with 2 models

In [20]:
df1a = df1[65000:]

In [21]:
df1a

,Unnamed: 0,index,pubdate,non_view_engagements,comment_count,like_count,dislike_count,love_count,haha_count,wow_count,...,cleanText_headline,cleanText_content,sentiment_headline,sentiment_label_headline,sentiment_content,sentiment_label_content,ESG_label_headline,ESG_label_score_headline,ESG_label_content,ESG_label_score_content
65000,65000,0,2021-07-02 18:29:00,0,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65001,65001,1,2021-07-02 18:42:00,0,NaN,0.0,0.0,0.0,0.0,0.0,...,HKET HOLDINGS( ) || Financial Statements/ESG I...,stock name: HKET HOLDINGS stock-code: title: E...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65002,65002,2,2021-07-02 18:37:00,0,NaN,0.0,0.0,0.0,0.0,0.0,...,HKET HOLDINGS( ) || Financial Statements/ESG I...,stock name: HKET HOLDINGS stock-code: title: A...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65003,65003,3,2021-07-02 18:58:00,0,NaN,0.0,0.0,0.0,0.0,0.0,...,Hermes SS22 is more environmentally friendly a...,2021-07-02 18:58 Different fashion designs suc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65004,65004,4,2021-07-02 00:00:00,0,NaN,0.0,0.0,0.0,0.0,0.0,...,Luo Bingxian Yongdao expects new shares to rai...,Luo Bingxian Yongdao expressed his optimism to...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,119995,54995,2021-04-16 18:30:00,8,NaN,7.0,0.0,NaN,NaN,NaN,...,"""Meteorological Cold Knowledge"": Strong Thunde...","""Meteorological Cold Knowledge"": Strong Thunde...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119996,119996,54996,2021-04-21 18:30:00,5,NaN,4.0,0.0,NaN,NaN,NaN,...,"[""Meteorological Cold Knowledge"": How far is t...","[""Meteorological Cold Knowledge"": How far is t...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119997,119997,54997,2021-04-30 18:30:00,0,NaN,0.0,0.0,NaN,NaN,NaN,...,"""Meteorological Cold Knowledge"": Thai-style cr...","""Meteorological Cold Knowledge"": Thai-style cr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119998,119998,54998,2021-05-03 18:30:03,2,NaN,1.0,0.0,NaN,NaN,NaN,...,"""Meteorological Cold Knowledge"": Wind Memorand...","""Meteorological Cold Knowledge"": Wind Memorand...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
import threading
import warnings
from tqdm import tqdm

warnings.simplefilter('ignore')

import concurrent.futures

# Set the number of threads to use
num_threads = 4

#config
start= 65001 
end= 65101
checkpt=50
pipe = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, max_length=512, truncation=True)

# Define the processing function
def process_row(num, pipe, labels):
  sent_head_output= pipe(df1["cleanText_headline"][num])[0]
  df1["sentiment_label_headline"][num]=labels[int(sent_head_output['label'][-1])]
  df1["sentiment_headline"][num]=sent_head_output['score']

  sent_cont_output= pipe(df1["cleanText_content"][num])[0]
  df1["sentiment_label_content"][num]=labels[int(sent_cont_output['label'][-1])]
  df1["sentiment_content"][num]=sent_cont_output['score']

  ESG_head_output= nlpESG(df1["cleanText_headline"][num])[0]
  df1["ESG_label_headline"][num]=ESG_head_output['label']
  df1["ESG_label_score_headline"][num]=ESG_head_output['score']

  ESG_cont_output= nlpESG(df1["cleanText_content"][num])[0]
  df1["ESG_label_content"][num]=ESG_cont_output['label']
  df1["ESG_label_score_content"][num]=ESG_cont_output['score']

# Create a thread pool and submit the tasks
with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
  tasks = {executor.submit(process_row, num, pipe, labels): num for num in tqdm(range(start+1, end+2))}
  
  # Iterate over the completed tasks and save the results
  for task in concurrent.futures.as_completed(tasks):
    num = tasks[task]
    if num % checkpt == 0 or num == end:
      df_senti_index = df1.iloc[num-i:num]
      df_senti_index.to_csv(f"df_senti_index{num-1}.csv")


100%|██████████| 101/101 [00:00<00:00, 802.17it/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "<ipython-input-20-82f7fb684739>", line 41, in <module>
    for task in concurrent.futures.as_completed(tasks):
  File "/usr/lib/python3.8/concurrent/futures/_base.py", line 244, in as_completed
    waiter.event.wait(wait_timeout)
  File "/usr/lib/python3.8/threading.py", line 558, in wait
    signaled = self._cond.wait(timeout)
  File "/usr/lib/python3.8/threading.py", line 302, in wait
    waiter.acquire()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-20-82f7fb684739>", line 45, in <module>
    df_senti_index.to_csv(f"df_senti_index{num-1}.csv")
  File "/usr/lib/python3.8/concurrent/futures/_base.py", line 644, in __exit__
    self.shutdown(wait=True)
  File "/usr/lib/python3.8/concurrent

TypeError: ignored

In [17]:
from tqdm import tqdm

# Set the number of threads to use
num_threads = 4

# Define the processing function
def process_row(num, pipe, labels):
  sent_head_output= pipe(df1["cleanText_headline"][num])[0]
  df1["sentiment_label_headline"][num]=labels[int(sent_head_output['label'][-1])]
  df1["sentiment_headline"][num]=sent_head_output['score']

  sent_cont_output= pipe(df1["cleanText_content"][num])[0]
  df1["sentiment_label_content"][num]=labels[int(sent_cont_output['label'][-1])]
  df1["sentiment_content"][num]=sent_cont_output['score']

  ESG_head_output= nlpESG(df1["cleanText_headline"][num])[0]
  df1["ESG_label_headline"][num]=ESG_head_output['label']
  df1["ESG_label_score_headline"][num]=ESG_head_output['score']

  ESG_cont_output= nlpESG(df1["cleanText_content"][num])[0]
  df1["ESG_label_content"][num]=ESG_cont_output['label']
  df1["ESG_label_score_content"][num]=ESG_cont_output['score']

# Create a thread pool and submit the tasks
with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
  # Create a progress bar
  with tqdm(total=end-start) as pbar:
    tasks = {executor.submit(process_row, num, pipe, labels): num for num in range(start+1, end+2)}
  
    # Iterate over the completed tasks and update the progress bar
    for task in concurrent.futures.as_completed(tasks):
      num = tasks[task]
      pbar.update()
      if num % checkpt == 0 or num == end:
        df1_senti_index = df1.iloc[num-i:num]
        df1_senti_index.to_csv(f"df1_senti_index{num-1}.csv")

0it [00:00, ?it/s]


In [16]:
import threading
import warnings
from tqdm import tqdm
import concurrent.futures

warnings.simplefilter('ignore')
num_threads = 4

#config
start= 650001 
end= 130001
checkpt=5000

pipe = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, max_length=512, truncation=True)

# Define the processing function
def process_row(num, pipeline, labels):
  sent_head_output= pipeline(df1["cleanText_headline"][num])[0]
  df1["sentiment_label_headline"][num]=labels[int(sent_head_output['label'][-1])]
  df1["sentiment_headline"][num]=sent_head_output['score']

  sent_cont_output= pipeline(df1["cleanText_content"][num])[0]
  df1["sentiment_label_content"][num]=labels[int(sent_cont_output['label'][-1])]
  df1["sentiment_content"][num]=sent_cont_output['score']

  ESG_head_output= nlpESG(df1["cleanText_headline"][num])[0]
  df1["ESG_label_headline"][num]=ESG_head_output['label']
  df1["ESG_label_score_headline"][num]=ESG_head_output['score']

  ESG_cont_output= nlpESG(df1["cleanText_content"][num])[0]
  df1["ESG_label_content"][num]=ESG_cont_output['label']
  df1["ESG_label_score_content"][num]=ESG_cont_output['score']

# Create a thread pool and submit the tasks
with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
  tasks = {executor.submit(process_row, num, pipeline, labels): num for num in range(start+1, end+2)}
  
  # Iterate over the completed tasks and save the results
  for task in concurrent.futures.as_completed(tasks):
    num = tasks[task]
    if num % checkpt == 0 or num == end:
      df_senti_index = df1.iloc[num-i:num]
      df_senti_index.to_csv(f"df_senti_index{num-1}.csv")

In [10]:
import multiprocessing

# Get the number of physical cores on the CPU
num_cores = multiprocessing.cpu_count()
num_cores

4

In [22]:
df1

,Unnamed: 0,index,pubdate,non_view_engagements,comment_count,like_count,dislike_count,love_count,haha_count,wow_count,...,cleanText_headline,cleanText_content,sentiment_headline,sentiment_label_headline,sentiment_content,sentiment_label_content,ESG_label_headline,ESG_label_score_headline,ESG_label_content,ESG_label_score_content
0,0,0,2020-09-10 11:56:07,1,0.0,1.0,0.0,0.0,0.0,0.0,...,"\ n . [We, never let Nangsheng encapsulate.]...",". [We, never let Nandosheng encapsulate.] One...",0.982091,Neutral,0.967710,Neutral,Pollution & Waste,0.290469,Pollution & Waste,0.254028
1,1,1,2020-05-21 00:23:03,0,0.0,0.0,0.0,0.0,0.0,0.0,...,"[HoneyBee] It turned out that today's ""World B...","[HoneyBee] It turned out that today's ""World B...",0.903570,positive,0.903570,positive,Natural Capital,0.931946,Natural Capital,0.931946
2,2,2,2021-09-07 14:31:02,47,4.0,41.0,0.0,NaN,NaN,NaN,...,Meet the Director of Anbai Global Finance and ...,Meet the director of Anbai Global Finance And ...,0.999968,Neutral,0.997915,Neutral,Community Relations,0.893131,Community Relations,0.936199
3,3,3,2020-07-20 16:56:49,0,NaN,0.0,NaN,0.0,0.0,0.0,...,nd 'S' of 6S Quality Process 'Sourcing' #6squa...,. [#Sustainability x NU SKIN 6s-Find the best...,0.998486,Neutral,0.986318,Neutral,Product Liability,0.369337,Pollution & Waste,0.941735
4,4,4,2020-07-20 21:14:41,0,NaN,0.0,NaN,0.0,0.0,0.0,...,The 3rd 'S' of 6S Quality Process 'Specificati...,The 3rd 's' of 6s Quality Process 'Specificati...,0.997296,Neutral,0.996695,Neutral,Product Liability,0.635044,Product Liability,0.691391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,119995,54995,2021-04-16 18:30:00,8,NaN,7.0,0.0,NaN,NaN,NaN,...,"""Meteorological Cold Knowledge"": Strong Thunde...","""Meteorological Cold Knowledge"": Strong Thunde...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119996,119996,54996,2021-04-21 18:30:00,5,NaN,4.0,0.0,NaN,NaN,NaN,...,"[""Meteorological Cold Knowledge"": How far is t...","[""Meteorological Cold Knowledge"": How far is t...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119997,119997,54997,2021-04-30 18:30:00,0,NaN,0.0,0.0,NaN,NaN,NaN,...,"""Meteorological Cold Knowledge"": Thai-style cr...","""Meteorological Cold Knowledge"": Thai-style cr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119998,119998,54998,2021-05-03 18:30:03,2,NaN,1.0,0.0,NaN,NaN,NaN,...,"""Meteorological Cold Knowledge"": Wind Memorand...","""Meteorological Cold Knowledge"": Wind Memorand...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
import warnings
warnings.simplefilter('ignore')

i = 0

#config
start= 64900 
end= 65000
checkpt=2000
pipe = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, max_length=512, truncation=True)

for num in tqdm(range(start+1, end+2)):
  sent_head_output= pipe(df1["cleanText_headline"][num-start])[0]
  df1["sentiment_label_headline"][num-start]=labels[int(sent_head_output['label'][-1])]
  df1["sentiment_headline"][num-start]=sent_head_output['score']

  sent_cont_output= pipe(df1["cleanText_content"][num-start])[0]
  df1["sentiment_label_content"][num-start]=labels[int(sent_cont_output['label'][-1])]
  df1["sentiment_content"][num-start]=sent_cont_output['score']

  ESG_head_output= nlpESG(df1["cleanText_headline"][num])[0]
  df1["ESG_label_headline"][num-start]=ESG_head_output['label']
  df1["ESG_label_score_headline"][num-start]=ESG_head_output['score']

  ESG_cont_output= nlpESG(df1["cleanText_content"][num-start])[0]
  df1["ESG_label_content"][num-start]=ESG_cont_output['label']
  df1["ESG_label_score_content"][num-start]=ESG_cont_output['score']

  if i % checkpt == 0 or num == end:
    df1_senti_index = df1.iloc[num-i:num]
    df1_senti_index.to_csv(f"df1_senti_index{num-1}.csv")
    i = 0
  i+=1

  4%|▍         | 4/101 [01:40<40:39, 25.15s/it]


KeyboardInterrupt: ignored